# price forcasting Notebook
This notebook loads data and prepares price prediction workflow.

In [37]:
import pandas as pd
import numpy as py

from google.colab import drive
drive.mount('/content/drive')

# Load CSV file from Google Drive
file_path = '/content/drive/MyDrive/fourth/project/wfp_food_prices_eth.csv'
df = pd.read_csv(file_path)

# Access a column
print(df.shape)
print(df.columns)

# View first 5 rows
print(df.head(8))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(40482, 14)
Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice'],
      dtype='object')
         date       admin1                             admin2  \
0       #date   #adm1+name                         #adm2+name   
1  2000-01-15  Addis Ababa                           AA ZONE1   
2  2000-01-15  Addis Ababa                           AA ZONE1   
3  2000-01-15  Addis Ababa                           AA ZONE1   
4  2000-01-15  Addis Ababa                           AA ZONE1   
5  2000-01-15       Amhara  Administrative unit not available   
6  2000-01-15    Dire Dawa                          DIRE DAWA   
7  2000-01-15       Tigray                             MEKELE   

             market   latitude  longitude            category  \
0  #loc+m

In [38]:
import pandas as pd
import numpy as np

# Drop first row, reset index, convert date column to datetime, and cast numeric columns
df = (
    df.drop(0)
      .reset_index(drop=True)
      .assign(date=lambda x: pd.to_datetime(x['date']))
      .astype({'price': float, 'usdprice': float})
)

# Sort by date, drop duplicates, reset index
df = df.sort_values('date').drop_duplicates().reset_index(drop=True)

# Extract year and month
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Clean commodity column: remove parentheses text and extract variety
df['commodity_base'] = df['commodity'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()
df['variety'] = df['commodity'].str.extract(r'\((.*?)\)', expand=False).fillna('')

# Check missing values
print(df.isnull().sum())

# Replace unavailable admin2 values with NaN
df['admin2'] = df['admin2'].replace('Administrative unit not available', np.nan)

# Convert latitude and longitude to numeric
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

# Keep only rows with positive price
df = df[df['price'] > 0].reset_index(drop=True)

# Convert categorical columns
df[['admin1', 'admin2', 'market', 'pricetype', 'category']] = df[['admin1', 'admin2', 'market', 'pricetype', 'category']].astype('category')

# Handle missing admin2 values
df['admin2'] = df['admin2'].cat.add_categories('Unknown').fillna('Unknown')

# Drop rows with missing admin1, latitude, longitude
df = df.dropna(subset=['admin1', 'latitude', 'longitude']).reset_index(drop=True)

# Replace empty variety with 'Standard'
df['variety'] = df['variety'].replace('', 'Standard').fillna('Standard')

# Clean unit column
df['unit'] = df['unit'].str.strip().str.upper()

# Drop unnecessary columns
df = df.drop(columns=['commodity','priceflag','currency'])

# Final checks
print(df.shape)
print(df.tail())
print(df.head())
print(df.info())

date              0
admin1            5
admin2            5
market            0
latitude          5
longitude         5
category          0
commodity         0
unit              0
priceflag         0
pricetype         0
currency          0
price             0
usdprice          0
year              0
month             0
commodity_base    0
variety           0
dtype: int64
(40476, 15)
            date   admin1  admin2             market  latitude  longitude  \
40471 2023-12-15  Gambela  ZONE 3  Kowerneng/Korgang  8.326659  33.953408   
40472 2023-12-15  Gambela  ZONE 2             Punido  7.749505  34.426848   
40473 2023-12-15  Gambela  ZONE 2             Punido  7.749505  34.426848   
40474 2023-12-15  Gambela  ZONE 2             Punido  7.749505  34.426848   
40475 2023-12-15   Somali    GODE               Gode  6.022500  43.130278   

                  category    unit pricetype    price  usdprice  year  month  \
40471   cereals and tubers  100 KG    Retail   4500.0   80.7609  2023   

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40476 entries, 0 to 40475
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            40476 non-null  datetime64[ns]
 1   admin1          40476 non-null  category      
 2   admin2          40476 non-null  category      
 3   market          40476 non-null  category      
 4   latitude        40476 non-null  float64       
 5   longitude       40476 non-null  float64       
 6   category        40476 non-null  category      
 7   unit            40476 non-null  object        
 8   pricetype       40476 non-null  category      
 9   price           40476 non-null  float64       
 10  usdprice        40476 non-null  float64       
 11  year            40476 non-null  int32         
 12  month           40476 non-null  int32         
 13  commodity_base  40476 non-null  object        
 14  variety         40476 non-null  object        
dtypes:

In [ ]:
# ============================================================
# ADVANCED PRICE FORECASTING (LIGHTGBM SAFE + TIME SPLIT)
# Commodity + Region Forecast
# ============================================================

import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

# Optional Models
try:
    from xgboost import XGBRegressor
    xgb_available = True
except:
    xgb_available = False

try:
    from lightgbm import LGBMRegressor
    lgbm_available = True
except:
    lgbm_available = False

# ============================================================
# 1️⃣ SORT DATA BY DATE
# ============================================================
df = df.sort_values('date')

# ============================================================
# 2️⃣ CLEAN COLUMN NAMES (SAFE FOR LIGHTGBM)
# ============================================================
df.columns = [re.sub('[^A-Za-z0-9_]+', '_', col) for col in df.columns]

# ============================================================
# 3️⃣ ENCODE CATEGORICAL / OBJECT COLUMNS
# ============================================================
cat_cols = ['admin1','admin2','market','commodity_base','variety','unit','pricetype']
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le

# ============================================================
# 4️⃣ TIME FEATURES
# ============================================================
df['month_sin'] = np.sin(2*np.pi*df['month']/12)
df['month_cos'] = np.cos(2*np.pi*df['month']/12)
df['year_trend'] = df['year'] - df['year'].min()
df['quarter'] = df['date'].dt.quarter

# ============================================================
# 5️⃣ LAG FEATURES (Commodity + Region)
# ============================================================
group_cols = ['commodity_base','admin1']

df['lag_1']  = df.groupby(group_cols)['usdprice'].shift(1)
df['lag_3']  = df.groupby(group_cols)['usdprice'].shift(3)
df['lag_6']  = df.groupby(group_cols)['usdprice'].shift(6)
df['lag_12'] = df.groupby(group_cols)['usdprice'].shift(12)

df['roll_mean_3'] = df.groupby(group_cols)['usdprice'].transform(lambda x: x.rolling(3).mean())
df['roll_std_3'] = df.groupby(group_cols)['usdprice'].transform(lambda x: x.rolling(3).std())

df['price_trend'] = df['lag_1'] - df['lag_6']
df['price_volatility'] = df['roll_std_3'] / df['roll_mean_3']

# ============================================================
# 6️⃣ REMOVE NA FROM LAGS
# ============================================================
df_model = df.dropna().reset_index(drop=True)

# ============================================================
# 7️⃣ FEATURES & TARGET
# ============================================================
features = [
    'admin1','admin2','market',
    'latitude','longitude',
    'commodity_base','variety','unit','pricetype',
    'month_sin','month_cos','quarter','year_trend',
    'lag_1','lag_3','lag_6','lag_12',
    'roll_mean_3','roll_std_3','price_trend','price_volatility'
]

# Include any nutrition/other one-hot columns
features += [c for c in df_model.columns if c.startswith('category_')]

X = df_model[features]
y = df_model['usdprice']

# ============================================================
# 8️⃣ CLEAN FEATURE NAMES AGAIN (SAFE)
# ============================================================
X.columns = [re.sub('[^A-Za-z0-9_]+', '_', col) for col in X.columns]

# ============================================================
# 9️⃣ TIME-BASED SPLIT (80% train, 20% test)
# ============================================================
split_index = int(len(df_model) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# ============================================================
# 🔟 MODELS
# ============================================================
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=300, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor()
}

if xgb_available:
    models["XGBoost"] = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

if lgbm_available:
    models["LightGBM"] = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.03,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

# ============================================================
# 1️⃣1️⃣ TRAIN & EVALUATE
# ============================================================
results = {}
all_preds = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    all_preds[name] = preds

    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    results[name] = mae

    print(f"\n{name}")
    print("MAE :", round(mae, 2))
    print("RMSE:", round(rmse, 2))

# ============================================================
# 1️⃣2️⃣ BEST MODEL
# ============================================================
best_name = min(results, key=results.get)
best_model = models[best_name]

print("\n🏆 BEST MODEL:", best_name)

# ============================================================
# 1️⃣3️⃣ FEATURE IMPORTANCE (if available)
# ============================================================
if hasattr(best_model, "feature_importances_"):
    importance = pd.Series(best_model.feature_importances_, index=X.columns)
    print("\n🔝 Top Features:")
    print(importance.sort_values(ascending=False).head(10))

# ============================================================
# 1️⃣4️⃣ NEXT PERIOD FORECAST
# ============================================================
future_price = best_model.predict(X_test.tail(1))[0]
print("\n📈 Next predicted price:", round(future_price,2))

# ============================================================
# 1️⃣5️⃣ VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt

# Actual vs Predicted (first 200 points)
plt.figure(figsize=(12,5))
plt.plot(y_test.values[:200], label='Actual')
plt.plot(all_preds[best_name][:200], label='Predicted')
plt.title(f"{best_name} Predictions vs Actual")
plt.legend()
plt.show()

# Feature Importance Plot (if exists)
if hasattr(best_model, "feature_importances_"):
    importance.nlargest(10).plot(kind='bar', figsize=(10,5))
    plt.title("Top 10 Important Features")
    plt.show()

# Price Trend + Next Forecast
plt.figure(figsize=(12,5))
plt.plot(df['date'], df['usdprice'], label='Historical')
plt.scatter(df['date'].iloc[-1], future_price, color='red', label='Next Forecast', s=100)
plt.title("Price Trend & Forecast")
plt.legend()
plt.show()

print("📊 Visualization Complete")


Linear Regression
MAE : 28.62
RMSE: 70.99
